In [2]:
import torch 
import torch.nn as nn
import os
import time
import importlib
import json
from collections import OrderedDict
import logging
import argparse
import numpy as np 
import random
import time
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torch.backends.cudnn
import torchvision.utils
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix

import torchvision.transforms as transforms
import torchvision

**SIMPLE VGG ARCHITECTURE **

In [3]:
architecture = [64 , 64 , -1 , 128 , 128 , -1 , 256 , 256, 256 , -1, 512, 512, 512 , -1 , 512 ,512, 512,-1]
class VGG(nn.Module):
   def __init__(self,in_channels=3,num_classes=10):
    super(VGG,self).__init__()
    self.in_channels = in_channels
    self.mai = self.main_architecture(architecture)
    self.fcs = nn.Sequential(nn.Linear(512,4096),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(4096,4096),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(4096,num_classes))                               
   def forward(self,x):
     x = self.mai(x)
     x = x.reshape(x.shape[0], -1)
     x = self.fcs(x)
     return x 
   def main_architecture(self,architecture):
            blocks = []
            in_channels = self.in_channels
            for layer in architecture:
                      if layer != -1 : 
                        out_channels = layer
                        blocks+=[nn.Conv2d(in_channels=in_channels,out_channels = out_channels , kernel_size = (3,3) , stride =(1,1) ,padding = (1,1) ),nn.BatchNorm2d(layer),nn.ReLU()]
                        in_channels = layer 
                      elif layer == -1 :
                        blocks+=[(nn.MaxPool2d(kernel_size=(2,2),stride= (2,2)))]
            return nn.Sequential(*blocks)

  

In [4]:
transform = transforms.Compose([transforms.Pad(4),transforms.RandomHorizontalFlip(),transforms.RandomCrop(32),transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
test_transform  = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=True,transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=False,transform=test_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=64, shuffle=False)


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [10]:
depth = 2
epochs = 5
batch_size = 256
base_lr = 0.001
lr_decay = 0.1
milestones = '[80, 120]'
device = "cuda"
num_workers = 3

model = VGG(in_channels = 3, num_classes = 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=base_lr)
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 50 == 0:
      print ("Epoch {}, Step {} Loss: {:.4f}".format(epoch+1, i+1, loss.item()))   

Epoch 1, Step 50 Loss: 2.2989
Epoch 1, Step 100 Loss: 2.3561
Epoch 1, Step 150 Loss: 2.2739
Epoch 1, Step 200 Loss: 2.2287
Epoch 1, Step 250 Loss: 2.1492
Epoch 1, Step 300 Loss: 2.1338
Epoch 1, Step 350 Loss: 2.1772
Epoch 1, Step 400 Loss: 2.0471
Epoch 1, Step 450 Loss: 1.9989
Epoch 1, Step 500 Loss: 2.0576
Epoch 1, Step 550 Loss: 1.9274
Epoch 1, Step 600 Loss: 1.8634
Epoch 1, Step 650 Loss: 1.8943
Epoch 1, Step 700 Loss: 1.8361
Epoch 1, Step 750 Loss: 1.8466
Epoch 2, Step 50 Loss: 1.7281
Epoch 2, Step 100 Loss: 1.9103
Epoch 2, Step 150 Loss: 1.7001
Epoch 2, Step 200 Loss: 1.8693
Epoch 2, Step 250 Loss: 1.7214
Epoch 2, Step 300 Loss: 1.8023
Epoch 2, Step 350 Loss: 1.9823
Epoch 2, Step 400 Loss: 1.6821
Epoch 2, Step 450 Loss: 1.5913
Epoch 2, Step 500 Loss: 1.6903
Epoch 2, Step 550 Loss: 1.8703
Epoch 2, Step 600 Loss: 1.5478
Epoch 2, Step 650 Loss: 1.5454
Epoch 2, Step 700 Loss: 1.6727
Epoch 2, Step 750 Loss: 1.6656
Epoch 3, Step 50 Loss: 1.5535
Epoch 3, Step 100 Loss: 1.5969
Epoch 3, St

In [11]:
model.eval

<bound method Module.eval of VGG(
  (mai): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, ke

In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy ( test images ) : {} %'.format(100 * correct / total))

Accuracy ( test images ) : 55.59 %


BETTER ACCYRACY CAN BE ACHIEVED BY TUNING HYPERPARAMETERS PROPERLY
